In [11]:
import numpy as np
import csv
import scipy.io

In [2]:
csvFile = "res.txt"

In [3]:
# read the csv file
vectors = []
with open(csvFile) as f:
    csvReader = csv.reader(f, delimiter = ',')
    for row in csvReader:
        vectors.append([float(e) for e in row])
        
print(type (vectors[0][0]),vectors[0])

<class 'float'> [-0.0309107, 0.108237, -0.993644, 0.99968, -0.0159194, -0.0196415, -1.63108]


In [4]:
vecs = np.array(vectors,dtype = np.float32)
print(vecs.shape)

(34, 7)


In [5]:
noramlFromCam = vecs[:,:3]
noramlFromPC = vecs[:,3:6]
print(noramlFromCam[0,:])
print(noramlFromPC[0,:])

[-0.0309107  0.108237  -0.993644 ]
[ 0.99968   -0.0159194 -0.0196415]


In [27]:
# check the norm of the normal vectors
for i in range(vecs.shape[0]):
    print(i, 'from cam', np.linalg.norm(noramlFromCam[i,:]), 'from PC', np.linalg.norm(noramlFromPC[i,:]))

0 from cam 0.9999995 from PC 0.99999964
1 from cam 1.0 from PC 1.0000001
2 from cam 0.9999999 from PC 0.9999996
3 from cam 0.99999946 from PC 1.0
4 from cam 0.9999995 from PC 1.0000001
5 from cam 0.9999996 from PC 0.9999998
6 from cam 0.9999998 from PC 1.0
7 from cam 0.99999946 from PC 0.9999998
8 from cam 0.9999996 from PC 1.0000001
9 from cam 1.0000002 from PC 0.9999997
10 from cam 1.0000004 from PC 1.0
11 from cam 0.99999946 from PC 0.9999994
12 from cam 0.9999997 from PC 0.99999964
13 from cam 0.9999997 from PC 0.9999999
14 from cam 1.0000001 from PC 1.0
15 from cam 0.9999999 from PC 1.0
16 from cam 1.0 from PC 0.9999999
17 from cam 0.9999997 from PC 0.99999964
18 from cam 0.99999964 from PC 0.9999995
19 from cam 1.0 from PC 1.0000002
20 from cam 1.0000001 from PC 0.9999997
21 from cam 1.0000005 from PC 1.0000006
22 from cam 0.9999996 from PC 1.0000001
23 from cam 0.99999964 from PC 1.0000001
24 from cam 1.0000001 from PC 0.9999999
25 from cam 1.0000002 from PC 0.99999994
26 from c

In [6]:
v_c = noramlFromCam[0,:]
v_l = noramlFromPC[0,:]
# reform the equation according to the notebook for one example
A_c = np.array([[0, -v_c[2], v_c[1]],
               [v_c[2], 0 , -v_c[0]],
               [-v_c[1], v_c[0], 0]], dtype = np.float32)
A_l = np.zeros((3,9), dtype=np.float32)
expanded = np.expand_dims(v_l,0)
A_l[0,0:3] = expanded
A_l[1,3:6] = expanded
A_l[2,6:9] = expanded
print("A_c:")
print(A_c)
print("A_l:")
print(A_l)
A = np.matmul(A_c,A_l)
print ("A: ")
print(A)

A_c:
[[ 0.         0.993644   0.108237 ]
 [-0.993644   0.         0.0309107]
 [-0.108237  -0.0309107  0.       ]]
A_l:
[[ 0.99968   -0.0159194 -0.0196415  0.         0.         0.
   0.         0.         0.       ]
 [ 0.         0.         0.         0.99968   -0.0159194 -0.0196415
   0.         0.         0.       ]
 [ 0.         0.         0.         0.         0.         0.
   0.99968   -0.0159194 -0.0196415]]
A: 
[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00  9.9332601e-01
  -1.5818216e-02 -1.9516658e-02  1.0820236e-01 -1.7230681e-03
  -2.1259370e-03]
 [-9.9332601e-01  1.5818216e-02  1.9516658e-02  0.0000000e+00
   0.0000000e+00  0.0000000e+00  3.0900808e-02 -4.9207982e-04
  -6.0713250e-04]
 [-1.0820236e-01  1.7230681e-03  2.1259370e-03 -3.0900808e-02
   4.9207982e-04  6.0713250e-04  0.0000000e+00  0.0000000e+00
   0.0000000e+00]]


In [13]:
# make the big A matrix in equation AR = 0
A = np.empty((0,9), dtype=np.float32)
for idx in range(vecs.shape[0]):
    v_c = noramlFromCam[idx,:]
    v_l = noramlFromPC[idx,:]
    A_c = np.array([[0, -v_c[2], v_c[1]],
                    [v_c[2], 0 , -v_c[0]],
                    [-v_c[1], v_c[0], 0]], dtype = np.float32)
    A_l = np.zeros((3,9), dtype=np.float32)
    expanded = np.expand_dims(v_l,0)
    A_l[0,0:3] = expanded
    A_l[1,3:6] = expanded
    A_l[2,6:9] = expanded
    a = np.matmul(A_c,A_l)
    A = np.concatenate((A,a),axis=0)
print (A.shape)
print (A[12:18,:])
scipy.io.savemat('A.mat',mdict={'A': A}) # to check the svd in Matlab as well.

(102, 9)
[[ 0.          0.          0.          0.9928119  -0.06490025  0.05722414
  -0.05956154  0.00389355 -0.00343303]
 [-0.9928119   0.06490025 -0.05722414  0.          0.          0.
  -0.05690531  0.00371991 -0.00327993]
 [ 0.05956154 -0.00389355  0.00343303  0.05690531 -0.00371991  0.00327993
   0.          0.          0.        ]
 [ 0.          0.          0.          0.9732081  -0.0228026   0.20040181
   0.06124817 -0.00143507  0.01261215]
 [-0.9732081   0.0228026  -0.20040181  0.          0.          0.
  -0.08905507  0.00208659 -0.01833811]
 [-0.06124817  0.00143507 -0.01261215  0.08905507 -0.00208659  0.01833811
   0.          0.          0.        ]]


In [8]:
# calculate the SVD and solve for R in AR = 0
u,s,v = np.linalg.svd(A)
print("v:", v.shape)
print(v)
print("s:", s.shape)
print(s)
print("u:", u.shape)
print(u)

v: (9, 9)
[[-9.50357735e-01  3.17152292e-02 -2.47063860e-01 -1.57164093e-02
   8.52811895e-03 -1.60221406e-03 -1.70825437e-01  8.35461356e-03
  -7.21137151e-02]
 [-2.76097190e-02 -2.46996749e-02 -8.21555872e-03  9.55237031e-01
  -5.37583977e-02  2.65684754e-01  6.50251657e-02  8.99107158e-02
   1.70799680e-02]
 [ 1.54861331e-01  4.47984086e-04 -7.67409205e-02  2.37647723e-02
  -6.39731467e-01  3.34591828e-02 -6.57906950e-01 -1.56778559e-01
  -3.19009304e-01]
 [ 5.09570502e-02  9.60146070e-01 -3.58240791e-02  2.97672488e-03
  -4.90612071e-03 -5.40395326e-04 -2.88785491e-02  2.69407988e-01
  -2.81376056e-02]
 [-1.94224939e-02  2.55955249e-01  4.81964797e-02  1.72580749e-01
   1.61120340e-01 -2.69090116e-01  2.02203114e-02 -8.95370424e-01
   3.91966179e-02]
 [ 1.49411827e-01 -9.57631320e-02 -4.20436502e-01  1.86173111e-01
   2.73006350e-01 -6.52692378e-01 -3.35516706e-02  2.06979141e-01
  -4.61055696e-01]
 [-1.62766978e-01 -2.03713682e-02  5.14809012e-01  1.34739220e-01
  -3.47332299e-01 

In [28]:
R_vec = v[-1,:]
print(R_vec)

[ 0.05541444 -0.03541288  0.22991571  0.06289712  0.58467335  0.0275417
 -0.72378     0.10461473  0.2479006 ]


In [29]:
# reshape to R
R = R_vec.reshape((3,3))
print(R)

[[ 0.05541444 -0.03541288  0.22991571]
 [ 0.06289712  0.58467335  0.0275417 ]
 [-0.72378     0.10461473  0.2479006 ]]


In [21]:
# Check how close it is to a rotation matrix now
def check(R):
    for i in range(3):
        n = np.linalg.norm(R[:,i])
        print("norm of column", i , "is" , n)

    print ("R*R.transpose= ")
    RRT = np.matmul(R,R.transpose())
    print (RRT)

check(R)

norm of column  0 is 0.728618
norm of column  1 is 0.5950137
norm of column  2 is 0.3392263
R*R.transpose= 
[[ 0.05718606 -0.01088729  0.01318367]
 [-0.01088729  0.34655753  0.02246937]
 [ 0.01318367  0.02246937  0.59625643]]


In [25]:
# try to turn it to a rotational Matrix
U,S,V = np.linalg.svd(R)
R_new = np.matmul(U, V.transpose())
print("S:",S)
print(R_new)

S: [0.77365094 0.5874115  0.23751189]
[[-0.35762963 -0.02971518  0.93339056]
 [ 0.10502882  0.9918725   0.0718189 ]
 [-0.9279386   0.12371748 -0.35160205]]


In [24]:
check(R_new)

norm of column  0 is 1.0
norm of column  1 is 1.0
norm of column  2 is 0.99999994
R*R.transpose= 
[[ 9.9999988e-01 -1.4845863e-08  1.0247673e-08]
 [-1.4845863e-08  1.0000001e+00 -6.5472765e-09]
 [ 1.0247673e-08 -6.5472765e-09  1.0000000e+00]]
